# Tensorboard使用
1. 指定logdir,创建SummaryWriter对象：writer = SummaryWriter("./logs/2_Tensorboard")
3. 添加数据
2. wirter.add_image()
3. writer.add_scalar()
4. writer.close()


控制台输入：tensorboard --logdir=logs --port=6006(可选)

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import os
from PIL import Image
import torch
from torchvision import transforms, utils
import math

In [ ]:
writer = SummaryWriter("./logs/2_Tensorboard")

In [ ]:
# 常用步骤为以下三步
# 1 2 添加
# writer.add_image()
# writer.add_scalar()
# 3 close
# writer.close()

In [ ]:
# 如何添加函数图像
# writer.add_scalar()
    # def add_scalar(
    #     self,
    #     tag,                图表title
    #     scalar_value,       图标数值 y轴
    #     global_step=None,   步数 x轴
    #     walltime=None,
    #     new_style=False,
    #     double_precision=False,
    # ):

for i in range(100):
    # 不同的tag会绘制到不同的表中，不用怕串
    # writer.add_scalar("y = x", i, i)
    writer.add_scalar("y = 2x", 2 * i, i)

In [ ]:
# 如何添加图像
# writer.add_image()
    # def add_image(
    #     self,
    #     tag,                 图title
    #     img_tensor,          图数据，需要：torch.Tensor, numpy.ndarray, or string/blobname 类型
    #     global_step=None,
    #     walltime=None,
    #     dataformats="CHW"。  img_tensor默认尺寸为(3, H, W)，3通道H高度W宽度。如果是其他形状需要使用dataformats变量来进行说明。
    # ):

In [ ]:
image_path = "./data/hymenoptera_data/train/ants"

In [ ]:
# 定义图像预处理变换：调整为统一大小并转换为Tensor格式
# 这一部分详细内容看3_Transforms
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # 调整图片大小
    transforms.ToTensor()           # 转换为Tensor
])

In [ ]:
# 获取 image_path 文件夹下的所有文件名
image_files = os.listdir(image_path)
# 用于存储所有图片的Tensor
images_list = []

In [ ]:
# 遍历所有文件，将图片打开、预处理后保存到列表中
for file_name in image_files:
    full_path = os.path.join(image_path, file_name)
    # 打开图片，并确保转换为RGB模式（确保一致性）
    img_pil = Image.open(full_path).convert("RGB")
    # 应用预处理变换，将PIL Image转换为Tensor
    img_tensor = transform(img_pil)
    images_list.append(img_tensor)

In [ ]:
# 将列表中所有的Tensor堆叠成一个Tensor，形状为 [N, C, H, W]
img = torch.stack(images_list)

In [ ]:
# 定义每个 global_step 下要显示的图片数量
img_num = 8  # 这里可以按需要调整

In [ ]:
# 总图片数
total_images = img.size(0)

In [ ]:
# 按 img_num 数量对图片进行分组，每个分组对应一个 global_step
num_steps = math.ceil(total_images / img_num)

In [ ]:
for step in range(num_steps):
    # 计算当前批次的起始和结束索引
    start_idx = step * img_num
    end_idx = min((step + 1) * img_num, total_images)

    # 从总图片中切出当前批次
    subset = img[start_idx:end_idx]

    # 生成图片网格, nrow 参数可以根据需要调整，这里简单使用 img_num 或计算一个较为合理的行数
    grid = utils.make_grid(subset, nrow=int(math.sqrt(img_num)))

    # 将当前图片网格写入 TensorBoard，并以当前 step 作为 global_step
    writer.add_image("All_Images", grid, global_step=step)

    print(f"写入 global_step = {step}, 图片索引范围：[{start_idx}, {end_idx})")

In [ ]:
writer.close()